In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import pandas as pd
import spacy


/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: Deprecation

In [2]:
nlp = spacy.load('en_core_web_sm')
sid = SentimentIntensityAnalyzer()
#CREATE DATAFRAME FROM THE FILE
df = pd.read_csv('amazon_reviews.txt', delimiter = '\t')

In [3]:
#CONVERT THE LABELS TO FAKE & NON FAKE
df.loc[df["LABEL"] == "__label1__", "LABEL"] = 'Fake'
df.loc[df["LABEL"] == "__label2__", "LABEL"] = 'Not Fake'

In [4]:
# Function to count nouns in a text
def count_nouns(Pos_counts):
    noun_count = Pos_counts.get(spacy.parts_of_speech.NOUN, 0)
    return noun_count

# Function to count verbs in a text
def count_verbs(Pos_counts):
    verb_count = Pos_counts.get(spacy.parts_of_speech.VERB, 0)
    return verb_count

# Function to count adjectives in a text
def count_adjectives(Pos_counts):
    adjective_count = Pos_counts.get(spacy.parts_of_speech.ADJ, 0)
    return adjective_count

# Function to count adverbs in a text
def count_adverbs(Pos_counts):
    adverb_count = Pos_counts.get(spacy.parts_of_speech.ADV, 0)
    return adverb_count

def pos_counts(text):
    doc = nlp(text)
    Pos_counts = doc.count_by(spacy.attrs.POS)
    return Pos_counts

In [5]:
df['POS_COUNTS'] = df['REVIEW_TEXT'].apply(pos_counts)
df['NUM NOUNS'] = df['POS_COUNTS'].apply(count_nouns)
df['NUM VERBS'] = df['POS_COUNTS'].apply(count_verbs)
df['NUM ADJECTIVES'] = df['POS_COUNTS'].apply(count_adjectives)
df['NUM ADVERBS'] = df['POS_COUNTS'].apply(count_adverbs)
df['REVIEW_LENGTH'] = df['REVIEW_TEXT'].apply(lambda d: len(d))
df['SENTIMENT SCORE'] = df['REVIEW_TEXT'].apply(lambda d: sid.polarity_scores(d)['compound'])
label_encoder = LabelEncoder()
df['VERIFIED_PURCHASE'] = label_encoder.fit_transform(df['VERIFIED_PURCHASE'])


KeyboardInterrupt: 

In [ ]:
df.to_csv('amazon_reviews.tsv', sep='\t', index=False)

In [ ]:
# Function to calculate the cosine similarity between two texts
def calculate_cosine_similarity(text1, text2):
    # Tokenize and convert the texts to spaCy Doc objects
    doc1 = nlp(text1)
    doc2 = nlp(text2)

    # Get the word vectors for each token in the documents
    vec1 = doc1.vector.reshape(1, -1)
    vec2 = doc2.vector.reshape(1, -1)

    # Calculate the cosine similarity between the vectors
    similarity = cosine_similarity(vec1, vec2)[0][0]
    return similarity

In [6]:
df = pd.read_csv('amazon_reviews.tsv', sep = '\t')
# Get random 1000 records
df1 = df.sample(n=1000, random_state=42)  # Make a copy of the top 1000 records as a DataFrame

In [7]:
df1.columns

Index(['eDOC_ID', 'LABEL', 'RATING', 'VERIFIED_PURCHASE', 'PRODUCT_CATEGORY',
       'PRODUCT_ID', 'PRODUCT_TITLE', 'REVIEW_TITLE', 'REVIEW_TEXT',
       'POS_COUNTS', 'NUM NOUNS', 'NUM VERBS', 'NUM ADJECTIVES', 'NUM ADVERBS',
       'REVIEW_LENGTH', 'SENTIMENT SCORE'],
      dtype='object')

In [ ]:
##ADD A MAXIMUM SIMILARITY COLUMN
tfidfvectoriser = T

In [27]:
print('hello')

hello


In [28]:
X = df1[['RATING', 'VERIFIED_PURCHASE', 'REVIEW_TEXT', 'NUM NOUNS', 'NUM VERBS','NUM ADJECTIVES', 'NUM ADVERBS', 'REVIEW_LENGTH', 'SENTIMENT SCORE']]
Y = df1['LABEL']

In [29]:
text_pipeline = Pipeline([
    ('selector', FunctionTransformer(
        lambda x: x['REVIEW_TEXT'], validate=False)),
    ('tfidf', TfidfVectorizer()),
])
numeric_pipeline = Pipeline([
    ('selector', FunctionTransformer(
        lambda x: x[['NUM NOUNS','NUM VERBS','NUM ADVERBS','NUM ADJECTIVES','VERIFIED_PURCHASE', 'RATING', 'REVIEW_LENGTH', 'SENTIMENT SCORE']], validate=False)),
    ('scaler', StandardScaler()),
])

feature_union = FeatureUnion([
    ('text_features', text_pipeline),
    ('numeric_features', numeric_pipeline)
])

In [30]:

# #GET THE SPLIT TRAINING AND TESTING DATA: 80-20
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X,Y, test_size=0.2, random_state=42) #RANDOMLY SELECT BECAUSE THEY ARE CONTINOUSLY ARRANGED


# # Fit and transform the training data
X_TRAIN_TRANSFORMED = feature_union.fit_transform(X_TRAIN, Y_TRAIN)


/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


In [31]:
print(df1)

       eDOC_ID     LABEL  RATING  VERIFIED_PURCHASE        PRODUCT_CATEGORY  \
2312      2313      Fake       3                  0                   Shoes   
16845    16846  Not Fake       4                  1             Electronics   
6105      6106      Fake       5                  0                      PC   
18082    18083  Not Fake       1                  1                    Toys   
11335    11336  Not Fake       2                  1               Furniture   
17677    17678  Not Fake       5                  1                 Kitchen   
1092      1093      Fake       4                  0         Lawn and Garden   
8364      8365      Fake       5                  0               Video DVD   
18550    18551  Not Fake       1                  1     Musical Instruments   
11423    11424  Not Fake       5                  1               Video DVD   
6757      6758      Fake       5                  0                    Toys   
12893    12894  Not Fake       5                  1 

In [32]:
# # Transform the testing data
X_TEST_TRANSFORMED = feature_union.transform(X_TEST)

print(Y)

2312         Fake
16845    Not Fake
6105         Fake
18082    Not Fake
11335    Not Fake
17677    Not Fake
1092         Fake
8364         Fake
18550    Not Fake
11423    Not Fake
6757         Fake
12893    Not Fake
17446    Not Fake
11823    Not Fake
4772         Fake
18882    Not Fake
7700         Fake
5018         Fake
1474         Fake
4966         Fake
6241         Fake
474          Fake
19606    Not Fake
3316         Fake
2714         Fake
8918         Fake
5701         Fake
5948         Fake
4059         Fake
19103    Not Fake
           ...   
6595         Fake
4309         Fake
3958         Fake
19404    Not Fake
16400    Not Fake
1272         Fake
3189         Fake
13293    Not Fake
20497    Not Fake
8887         Fake
1475         Fake
2533         Fake
13674    Not Fake
12004    Not Fake
14480    Not Fake
3836         Fake
13188    Not Fake
3472         Fake
5735         Fake
16532    Not Fake
20188    Not Fake
606          Fake
17601    Not Fake
2826         Fake
13925    N

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [33]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_TRAIN_TRANSFORMED, Y_TRAIN)

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [34]:
predictions = clf.predict(X_TEST_TRANSFORMED)
print(classification_report(Y_TEST, predictions))

              precision    recall  f1-score   support

        Fake       0.87      0.69      0.77        94
    Not Fake       0.77      0.91      0.83       106

   micro avg       0.81      0.81      0.81       200
   macro avg       0.82      0.80      0.80       200
weighted avg       0.81      0.81      0.80       200



/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/linear_model/base.py:283: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [ ]:
df1['MAX_SIMILARITY'] = df1['REVIEW_TEXT'].apply(lambda x: max(df1.loc[df1['REVIEW_TEXT'] != x, 'REVIEW_TEXT'].apply(lambda y: calculate_cosine_similarity(x, y))))